# <font color='blue'>Acai CRM Sales Analysis :</font>

#### - Kinsuk Ghatak
#### 9038426200
#### kinsukghatak@gmail.com 


## <font color='red'>A. Exploratory and Descriptive analysis on the transaction data set </font> :

In [ ]:
## EDA Codes : 
import pandas as pd
import numpy as np
import matplotlib as plt

df_acai=pd.read_excel("C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/master_data_acai.xlsx")
df_country=pd.read_excel("C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/country_code.xlsx")

df_acai=df_acai.merge(df_country.rename({'Country_Name': 'Shipping Country Name'}, axis=1),
              left_on='Shipping Country',right_on='Country_Code',how='left')

df_acai=df_acai.merge(df_country.rename({'Country_Name': 'Billing Country Name'}, axis=1),
              left_on='Billing Country',right_on='Country_Code',how='left')


print("length of the data set :",len(df_acai))
print("Total Columns of the data set :",len(df_acai.columns))
print("# of unique customers in the entire tenure from 2016 to 2021 :",len(df_acai.username.unique()))



In [ ]:
##Pre processing and MVI for rolling up information at transaction level :

from datetime import datetime
import calendar
print(df_acai['Created at'].dtypes)

from pandas.api.types import CategoricalDtype

cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
cat_type = CategoricalDtype(categories=cats, ordered=True)

cats_month = [ 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul','Aug','Sep','Oct','Nov','Dec']
cats_month_type = CategoricalDtype(categories=cats_month, ordered=True)

cats_year = [ '2016','2017','2018','2019','2020','2021']
cats_year_type = CategoricalDtype(categories=cats_year, ordered=True)



df_acai['Total'] = pd.to_numeric(df_acai['Total'],errors = 'coerce')


df_acai['date_string']=df_acai['Created at'].str[:10]

df_acai['Date']  = pd.to_datetime(df_acai['date_string'])

# df_acai['month_year'] = df_acai['Date'].dt.strftime('%b-%Y')
df_acai['month'] = df_acai['Date'].dt.strftime('%b').astype(cats_month_type)
df_acai['year'] = df_acai['Date'].dt.strftime('%Y').astype(cats_year_type)
df_acai['month_year']=df_acai['month'].astype(str)+'-'+df_acai['year'].astype(str)

df_acai['Day'] = df_acai['Date'].dt.strftime('%A').astype(cat_type)

df_acai=df_acai.drop(columns='date_string').copy()




## missing value filling up :
vars_ffill= ['Financial Status','Accepts Marketing','Shipping Method','Payment Method','Risk Level','Source',
             'Shipping Country','Billing Country']
for col in vars_ffill:
    df_acai[col]=df_acai[col].ffill()
    




# df_acai.to_csv('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/Processed_Sales_Data.csv')

print("Total number of unique SKUs %d : "  % len(df_acai['Lineitem name'].unique()))
print("Total number of unique Lineitem description %d : "  % len(df_acai['Lineitem name'].unique()))
print("Total number of unique Customers %d : "  % len(df_acai['Name'].unique()))


df_acai.head(2)


In [ ]:
#Variables to keep for analysis :
var_list = ['Name','username','year','month_year','month','Date','Day','Created at','Financial Status','Accepts Marketing','Shipping Method','Payment Method','Risk Level','Source',
            'Lineitem quantity','Total','Discount Code','Discount Amount','Lineitem price','Billing City','Billing Zip','Billing Province','Billing Country',
           'Billing Country Name','Shipping City','Shipping Zip','Shipping Province','Shipping Country','Shipping Country Name',
            'Created at' , 'Lineitem name','Lineitem sku','Product_Category']

In [ ]:
## Product segment processing and identification : 


keywords=['trouser','vest','bra','legging','tee','top','sweater','capri','shorts','socks','pants','thermal',
             'jeans','wash','care','hoodie','belt','sweatshirt','khaki','tights']

pat = '|'.join(r"\b{}\b".format(x) for x in keywords)

df_acai['Category1'] = df_acai['Lineitem name'].str.lower().str.findall(pat).apply(set).str.join(', ')
df_acai['Category2'] = df_acai['Lineitem name'].str.lower().str.findall('|'.join(keywords)).apply(set).str.join(', ')

df_acai['Product_Category']=df_acai['Category2'].apply(lambda x: x.split(',')[0])

# df_acai.loc[df_acai["Product_Category"].isnull(),'Product_Category'] = "Others"

df_acai.loc[df_acai.Product_Category == '', 'Product_Category'] = 'Others'

df_acai.head()

df_filtd_colmns=df_acai[var_list].copy()
df_filtd_colmns = df_filtd_colmns[df_filtd_colmns['year']!='2016']

print(len(df_filtd_colmns))


# df_filtd_colmns.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/Processed_Sales_Data_v3.xlsx')

In [ ]:
## Filtering only those transactions which has some status like paid / refunded etc. We have total 95484 such results and 
# a frequency table of the same is provided below to better understand the payment sceanario. 

df_filt=df_filtd_colmns[(df_filtd_colmns['Financial Status'].notnull())]
print(len(df_filt))
pd.crosstab(df_filtd_colmns['Financial Status'], columns='count')

In [ ]:
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context("paper", rc={"font.size":12,"axes.titlesize":15,"axes.labelsize":15})
sns.set_style('white')




df_filtd_colmns['year-month']=df_filtd_colmns['Date'].dt.to_period('M')


df_filtd_colmns['Lineitem_name_processed']=df_filtd_colmns['Lineitem name'].str.lower().apply(lambda x: x.strip())




df_sku_month_yr = df_filtd_colmns.groupby(['year','year-month','month']).agg({'Lineitem_name_processed': 'nunique'}).reset_index().rename(columns={'Lineitem_name_processed': 'Unique counts of SKUs'})

# df_sku_month_yr.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/sku_analysis.xlsx')


sns.barplot(y='Unique counts of SKUs', x='year-month',data=df_sku_month_yr)
plt.xticks(rotation=60)


plt.xticks(rotation=60)

plt.title('Month on month change of SKU',fontsize=15,color="black",fontweight='bold')
plt.show()


df_sku_yr = df_filtd_colmns.groupby(['year']).agg({'Lineitem_name_processed': 'nunique'}).reset_index().rename(columns={'Lineitem_name_processed': 'Unique counts of SKUs'})

df_sku_yr

In [ ]:
## Find unique increase of SKU counts : 

df_filtd_colmns = df_filtd_colmns[df_filtd_colmns['year']!='2016']

print("Total number of unique SKUs %d : "  % len(df_filtd_colmns['Lineitem name'].unique()))


print("Total number of unique SKUs in lower font %d : "  % len(df_filtd_colmns['Lineitem name'].str.lower().apply(lambda x: x.strip()).unique()))


df_filtd_colmns['Lineitem_name_processed']=df_filtd_colmns['Lineitem name'].str.lower().apply(lambda x: x.strip())


df_filtd_colmns['SKU_Code']=pd.factorize(df_filtd_colmns['Lineitem_name_processed'])[0]+1

df_sku_max_yr= df_filtd_colmns.groupby(['year']).agg({'SKU_Code':'max'}).reset_index().\
rename(columns={'SKU_Code':'Max_sku_count_year'})

df_sku_max_mth= df_filtd_colmns.groupby(['year-month']).agg({'SKU_Code':'max'}).reset_index().\
rename(columns={'SKU_Code':'Max_sku_count_mth'})

df_sku_max_yr['New Launches year']= df_sku_max_yr['Max_sku_count_year']- df_sku_max_yr['Max_sku_count_year'].shift(1)


df_sku_max_mth['New Launches month']= df_sku_max_mth['Max_sku_count_mth']- df_sku_max_mth['Max_sku_count_mth'].shift(1)

df_sku_max_mth.head(2)

df_sku_max_yr

df_sku_max_mth


In [ ]:
print ("Total numbr of unique SKS %d:" % len(df_filtd_colmns['Lineitem name'].str.lower().apply(lambda x: x.strip()).unique()))

print ("Total numbr of unique customers %d:" % len(df_filtd_colmns['username'].str.lower().unique()))


print ("Total numbr of unique transactions %d:" % len(df_filtd_colmns['Name'].str.lower().unique()))

In [ ]:
## Sales Movement and trends on month and weekdays : 

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context("paper", rc={"font.size":15,"axes.titlesize":25,"axes.labelsize":25})  
sns.set(font_scale = 1.4)
sns.set_style('white')

df_full= df_filtd_colmns[df_filtd_colmns['Financial Status']=='paid']

cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

cats_month = [ 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul','Aug','Sep','Oct','Nov','Dec']
cats_month_type = CategoricalDtype(categories=cats_month, ordered=True)

df_sales = df_full.groupby(['year','month_year','month','Day']).agg({'Total':'sum','Lineitem quantity':'sum'}).reset_index().rename(columns={'Total': 'Sales Value','Lineitem quantity': 'Sales Volume'})
# df_sku_month_yr=df_sku_month_yr[df_sku_month_yr['year']!='2016'].copy()

df_sales_day=df_full.groupby(['Day']).agg({'Total':'sum','Lineitem quantity':'sum'}).reset_index().\
rename(columns={'Total': 'Sales Value','Lineitem quantity': 'Sales Volume'}) 

df_sales_month=df_full.groupby(['month']).agg({'Total':'sum','Lineitem quantity':'sum'}).reset_index().\
rename(columns={'Total': 'Sales Value','Lineitem quantity': 'Sales Volume'}) 

sns.barplot(y='Sales Value', x='Day',data=df_sales_day).set_title('Weekday wise sales value',\
                                                                  fontdict= { 'fontsize': 20, 'fontweight':'bold'})
plt.show()

sns.barplot(y='Sales Value', x='month',data=df_sales_month).set_title('Month wise sales value',\
                                                                  fontdict= { 'fontsize': 20, 'fontweight':'bold'})
plt.show()

df_sales_month

### Customer retention and basket analysis : 

In [ ]:

##Customer retention analysis : 

df_username =df_acai[df_acai['year'] !='2016']

df_username=df_username[df_username['username'].notnull()]

df_username['year_numeric']=df_username['year'].astype(int)
df_username.head()


for year in (2016,2017,2018,2019,2020,2021):
    next_username_list= []
    prev_username_list =[]
    common_users_set =set()
    
    year_prev_str=year
    year_next_str =year+1
    
    df_prev= df_username[df_username['year_numeric']<=year_prev_str]
    prev_username_list=df_prev['username'].str.lower().unique().tolist()
    
    df_next = df_username[df_username['year_numeric']==year_next_str]
    next_username_list=df_next['username'].str.lower().unique().tolist()
    
    common_users_set = set(next_username_list) & set(prev_username_list)

    print("Total number of unique customers for ", year_next_str , "is %d:" %len(next_username_list))

    print ("Common Customers between year :",year_prev_str , 'and' ,year_next_str , 'is %d: '  %len(common_users_set))
    
print("Total number of unique customers for the period : %d " %len(df_username['username'].str.lower().unique()))


In [ ]:
##Basket Distribution :
## Customer wise finding no of qtys. :

import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context("paper", rc={"font.size":15,"axes.titlesize":25,"axes.labelsize":25})  
sns.set(font_scale = 1.4)
# sns.set_style('white')
sns.set_style("whitegrid", {'axes.grid' : False})

import matplotlib.pyplot as plt

df_filtd_colmns['Lineitem_name_processed']=df_filtd_colmns['Lineitem name'].str.lower().apply(lambda x: x.strip())

df_full= df_filtd_colmns[df_filtd_colmns['Financial Status']=='paid']


df_basket=df_full.groupby(['Name']).agg({'Lineitem_name_processed': pd.Series.nunique}).reset_index().rename(columns={'Lineitem_name_processed': 'Unique Products Bought'})
# df_basket.head()
df_product_basket_count=df_basket.groupby('Unique Products Bought').agg({'Name':'count'}).reset_index().rename(columns={'Name': 'Total_occurance'})
df_product_basket_count['Percentage Occurance'] = round(100*df_product_basket_count['Total_occurance']/df_product_basket_count['Total_occurance'].sum(),2)
# df_product_basket_count

sns.set_style('white')
sns.set_style('white')
sns.set(font_scale = 1.5)
sns.barplot(y='Percentage Occurance', x='Unique Products Bought',data=df_product_basket_count).set_title('Product Basket Analysis',
                                                                                                  fontdict= { 'fontsize': 20, 'fontweight':'bold'})


plt.show()



In [ ]:

##User vs # of transastions distribution


import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context("paper", rc={"font.size":15,"axes.titlesize":15,"axes.labelsize":15})   
import matplotlib.pyplot as plt


df_filtd_colmns['Lineitem_name_processed']=df_filtd_colmns['Lineitem name'].str.lower().apply(lambda x: x.strip())

df_full= df_filtd_colmns[df_filtd_colmns['Financial Status']=='paid']
df_full=df_full[df_full['username'].notnull()]
df_full['username']=df_full['username'].astype(str)


df_basket=df_full.groupby(['username']).agg({'Name': pd.Series.nunique}).reset_index().rename(columns={'Name': '#Transactions'})

df_customer_basket_count=df_basket.groupby('#Transactions').agg({'username':'count'}).reset_index().rename(columns={'username': 'Total_occurance'})
df_customer_basket_count['Percentage Occurance'] = round(100*df_product_basket_count['Total_occurance']/df_product_basket_count['Total_occurance'].sum(),2)
# df_customer_basket_count

df_customer_basket_count

# sns.set_style('white')
# sns.set(font_scale = 1.5)
# sns.barplot(y='Percentage Occurance', x='#Transactions',data=df_customer_basket_count).set_title('Product Basket Analysis',
#                                                                                                   fontdict= { 'fontsize': 20, 'fontweight':'bold'})


# plt.show()

In [ ]:
#Product Category analysis : 
import matplotlib.pyplot as plt
df_full= df_filtd_colmns[df_filtd_colmns['Financial Status']=='paid']


df_product =df_full.groupby(['Product_Category']).agg({'Total': 'sum', 'Lineitem quantity': 'sum'}).reset_index()
df_product.rename(columns = {'Total':'Total Sales Value GBP','Lineitem quantity':'Total Sales Qty'}, inplace = True)


df_product['perc_sales_val'] = round(100*df_product['Total Sales Value GBP']/df_product['Total Sales Value GBP'].sum(),2)
df_product['perc_sales_vol'] = round(100*df_product['Total Sales Qty']/df_product['Total Sales Qty'].sum(),2)

df_product_sorted=df_product.sort_values(by=['perc_sales_val'],ascending=False)
df_product_sorted['Cumulative_sales_val_prct']=df_product_sorted['perc_sales_val'].cumsum()
df_product_sorted['Cumulative_sales_vol_prct']=df_product_sorted['perc_sales_vol'].cumsum()

# print("Total number of unique discount strategies %d : "  % len(cross_rev_disc['Discount Code'].unique()))

df_product_sorted['Pie_Chart_Categ'] = np.where(df_product_sorted['Cumulative_sales_val_prct'] > 90 ,"Other", df_product_sorted['Product_Category'])
df_product_sorted.head(15)

df_product_pie =df_product_sorted.groupby(['Pie_Chart_Categ']).agg({'Total Sales Value GBP': 'sum'})

df_product_pie.plot.pie(y='Total Sales Value GBP',figsize=(10, 10),legend=False,autopct='%1.1f%%',\
                        startangle=0,textprops={'fontsize': 14})
plt.title('Major Category wise revenue distribution',fontsize=20,color="black",fontweight='bold')
plt.show()

## Product category wise unique counts  of SKU :


df_prd_sku=df_full.groupby(['Product_Category']).agg({'Lineitem name': pd.Series.nunique}).reset_index().rename(columns={'Lineitem name': 'Unique SKUS Count'})
df_prd_sku

In [ ]:
## Marketing and discounting effectiveness analysis :

from matplotlib import pyplot
df_discount= df_full.groupby(['Name']).agg({'Total': 'sum','Lineitem quantity' : 'sum','Discount Amount':'sum'}).reset_index().\
rename(columns={'Total': 'Sales Value','Lineitem quantity' : 'Sales Volume'})
df_discount.head()

sns.relplot(x='Discount Amount', y='Sales Value',data=df_discount)
sns.relplot(x='Discount Amount', y='Sales Volume',data=df_discount)

# corr=np.corrcoef(df_discount['Discount Amount'], df_discount['Total'])
# print('Pearsons correlation: %.3f' % corr)

df_discount.corr()

In [ ]:
## Geography wise analysis : 
pd.crosstab(index=df_full['Shipping Country'], columns=df_full['Billing Country']).apply(lambda r: r*100/r.sum(), axis=1)

##Since in majority of the cases billin=-Shipping ; we will proceed with billing country for country wise splits

### Price Bracket Analysis : 

In [ ]:
## Price brakcet distribution :

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context("paper", rc={"font.size":15,"axes.titlesize":25,"axes.labelsize":25})  
sns.set(font_scale = 1.4)
sns.set_style('white')


df_full['Price Bracket Range']=pd.cut(df_full['Lineitem price'],5)
df_full['Price Bracket bin']=pd.cut(df_full['Lineitem price'],5,labels=False)

price_bracket_pivot=df_full.groupby(['Price Bracket bin','Price Bracket Range']).agg({'Name':'count','Total':'sum'}).reset_index()\
.rename(columns={'Name':'Count of customers','Total':'Revenue Generated'})

price_bracket_pivot['Percentage of customers'] = round(100*price_bracket_pivot['Count of customers']/price_bracket_pivot['Count of customers'].sum(),2)
price_bracket_pivot['Percentage of revenue'] = round(100*price_bracket_pivot['Revenue Generated']/price_bracket_pivot['Revenue Generated'].sum(),2)
price_bracket_pivot.head()


ax = sns.barplot(x="Price Bracket bin", y="Percentage of customers", data=price_bracket_pivot)
ax.set(ylabel="Percent")

price_bracket_pivot.head()

print(price_bracket_pivot)
print(df_full['Lineitem price'].describe())

plt.xticks(rotation=60)
plt.show()
price_bracket_pivot


##Category wise price analysis : 

df_price_cat = df_full.groupby('Product_Category').agg({'Lineitem price':[np.mean, np.median,np.max,np.min]})
df_price_cat


In [ ]:
## Heatmap generettion to observe Category wise price brackets :

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context("paper", rc={"font.size":15,"axes.titlesize":25,"axes.labelsize":35})  
sns.set(font_scale = 1.5)
sns.set_style('white')

## Price brakcet distribution by product category :

func = lambda x: 100*x.count()/df_filtd_colmns.shape[0]

df_filtd_colmns['Price Bracket Range']=pd.cut(df_filtd_colmns['Lineitem price'],5)
df_filtd_colmns['Price Bracket bin']=pd.cut(df_filtd_colmns['Lineitem price'],5,labels=False)


price_bracket_pivot_product = df_filtd_colmns.pivot_table(values='Name',index='Product_Category',columns='Price Bracket bin',aggfunc=func)
price_bracket_pivot_product

sns.heatmap(price_bracket_pivot_product,annot=True,cmap = 'YlGnBu',cbar_kws={'label': 'Figs in %age'}).set_title('Category wise price brackets',\
                                                                    fontdict= { 'fontsize': 20, 'fontweight':'bold'})
plt.show()


In [ ]:
## Postal code wise sales value analysis :

import folium
import pgeocode


df_full_uk=df_full[(df_full['Shipping Country'] == "GB")]
df_sales_zip_uk= df_full_uk.groupby('Shipping Zip').agg({'Name': lambda qty: qty.count()}).sort_values(by=['Name'],ascending=False).reset_index()

nomi = pgeocode.Nominatim("gb")

dfzip = df_sales_zip_uk["Shipping Zip"].apply(lambda x:nomi.query_postal_code(x))
dfzip_final_uk = pd.concat([df_sales_zip_uk, dfzip], axis="columns")
dfzip_final_uk.head()
#for each row in the df_sales_zip dataset, plot the corresponding latitude and longitude on the map


In [ ]:
## Creation of heatmap for Great Britain : 

from folium import plugins
from folium.plugins import HeatMap

dfzip_final_uk_NoNaN=dfzip_final_uk[(dfzip_final_uk['latitude'].notnull()) & (dfzip_final_uk['longitude'].notnull())]

map_uk = folium.Map(location=[51.5074, 0.1278], zoom_start = 5) 


for i,row in dfzip_final_uk_NoNaN.iterrows():
    folium.CircleMarker((row.latitude,row.longitude), radius=3, weight=2, color='red', fill_color='red', fill_opacity=.5).add_to(map_uk)

    
map_uk.add_child(plugins.HeatMap(data=dfzip_final_uk_NoNaN[['latitude', 'longitude','Name']].as_matrix(), radius=15, blur=5))

# save the map as an html
map_uk.save('map_uk_heatmap.html')
map_uk



# <font color='red'>B. RFM Segmentation analysis for customer segementation :  </font>

In [ ]:
df_filtd_colmns['username_proscsd']=df_filtd_colmns['username'].str.lower()

df_filt_user=df_filtd_colmns[df_filtd_colmns['username_proscsd'].notnull()]


df_seg= df_filt_user[df_filt_user['Financial Status']=='paid']

df_user=df_seg.groupby(['username_proscsd']).agg({'Name':pd.Series.nunique}).rename(columns={'Name' : '#Transactions'}).reset_index()


df_seg2=df_seg.merge(df_user,on='username_proscsd',how='left')

df_segment_1=df_seg2[df_seg2['#Transactions']==1]
df_segment_grt_1=df_seg2[df_seg2['#Transactions']>=2]

print(len(df_seg))
print(len(df_seg2))
print(len(df_segment_1))
print(len(df_segment_grt_1))

In [ ]:
##Calculate R,F,M Values : 

from datetime import date
today = datetime.now()
df_segment_grt_1['Date_copy']=df_segment_grt_1['Date']

rfm_df= df_segment_grt_1.groupby('username_proscsd').agg({'Date': lambda date: (today - date.max()).days,
                                        'Date_copy': lambda Date: Date.nunique(),
                                        'Total': lambda Total: Total.sum()})

rfm_df.columns=['recency','frequency','monetary']

# rfm_df['monetary']=rfm_df['monetary']/rfm_df['frequency']

rfm_df.head()

In [ ]:
# applying RFM method for segmentation  : 

from datetime import date
import datetime as dt
today = datetime.now()


# df_segment_grt_1['Date_copy']=df_segment_grt_1['Date']

rfm_df= df_segment_grt_1.groupby('username_proscsd').agg({'Date': lambda date: (today - date.max()).days,
                                        'Name': lambda cust_id: cust_id.nunique(),
                                        'Total': lambda Total: Total.sum()})

rfm_df.columns=['recency','frequency','monetary']

# rfm_df['monetary_value']=rfm_df['monetary_total']/rfm_df['frequency']

# rfm_df.reset_index()

rfm_df=rfm_df.dropna()

#normalisation of the RFM columns to bring erverything at the same level : 
# Import library
from sklearn.preprocessing import StandardScaler
# Initialize the Object
scaler = StandardScaler()
# Fit and Transform The Data
scaler.fit(rfm_df)
rfm_normalized = scaler.transform(rfm_df)
 

rfm_df_normalized = pd.DataFrame(rfm_normalized, index=rfm_df.index, columns=rfm_df.columns)
rfm_df_normalized.head()

In [ ]:
# Plot RFM distributions
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12,10))# Plot distribution of R
plt.subplot(3, 1, 1); sns.distplot(rfm_df['recency'])# Plot distribution of F
plt.subplot(3, 1, 2); sns.distplot(rfm_df ['frequency'])# Plot distribution of M
plt.subplot(3, 1, 3); sns.distplot(rfm_df ['monetary'])# Show the plot
plt.show()


In [ ]:
## Box plot for Recency at an over all level : 
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context("paper", rc={"font.size":15,"axes.titlesize":25,"axes.labelsize":35})  
sns.set(font_scale = 1.5)
sns.set_style('white')

sns.boxplot(y="recency", data=rfm_df)
plt.title("Distribution of Recency accross all customers who bought more than once")


In [ ]:
## To analyse and reach optimum number of clusters using k means : 

from sklearn.cluster import KMeans
sse = {}
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_df_normalized.dropna())
    sse[k] = kmeans.inertia_ # SSE to closest cluster centroid
plt.title('The Elbow Method')
plt.xlabel('k_No. of clusters')
plt.ylabel('SSE')
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()

#### Post completing the elbow curve it seems that ther curve reaches it's inflection point at K=4 ; i.e post k=4 the rate of change or decrease of SSE w.r.t newer addition of clusters is not that significant and hence we finalise 4 as our final number of segments. 

In [ ]:
## Fitting the model with k=7 clusters on the rfm dataset : 

model = KMeans(n_clusters=7, random_state=42)
model.fit(rfm_df_normalized.dropna())
model.labels_.shape

rfm_df["kmeans_cluster"] = model.labels_
rfm_df.groupby('kmeans_cluster').agg({
    'recency':'mean',
    'frequency':'mean',
    'monetary':['sum','mean','median', 'count']}).round(2)

In [ ]:
## Merging all the customer segment information to the original data set : 

rfm_df=rfm_df.reset_index()
rfm_name_clust=rfm_df[['username_proscsd','kmeans_cluster']]
rfm_name_clust.head()

df_segment_full=df_segment_grt_1.merge(rfm_name_clust, on='username_proscsd', how='left')

df_segment_full.head()
df_segment_full.groupby('kmeans_cluster').agg({
    'username_proscsd':'count'}).round(2)

## Scoring based on R,F,M percentiles : 

In [ ]:

def RScore(x,p,d):
    if x <= d[p][0.20]:
        return 5
    elif x <= d[p][0.40]:
        return 4
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 2
    else:
        return 1

    
def FMScore(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 4
    else:
        return 5

In [ ]:
quantiles = rfm_df.quantile(q=[0.20,0.40,0.60,0.80,1])
quantiles

In [ ]:
##Calculate : R , F, M percentile values based on the relative positiosn and alsao create a combined score : 

rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['recency'].apply(RScore, args=('recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['frequency'].apply(FMScore, args=('frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['monetary'].apply(FMScore, args=('monetary',quantiles,))

rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()


In [ ]:
## Distribution analysis of RFM Summed up score : 

df_rfm_segmented=df_segment_grt_1.merge(rfm_segmentation,on='username_proscsd',how='left')


# Calculate RFM_Score
df_rfm_segmented['RFM_Score_sum'] = df_rfm_segmented[['R_Quartile','F_Quartile','M_Quartile']].sum(axis=1)


df_rfm_segmented.head()

sns.distplot(df_rfm_segmented['RFM_Score_sum'])
plt.show()

print(df_rfm_segmented['RFM_Score_sum'].describe())

quantiles_rfm = df_rfm_segmented['RFM_Score_sum'].quantile(q=[0.20,0.40,0.60,0.80,1])
quantiles_rfm

In [ ]:
# Define rfm_level function
## We will segment the customers into different buckets based on their final RFM scummed up score ranges :

def rfm_level(df):
    if df['RFM_Score_sum'] >= 14:
        return 'Can\'t Loose Them'
    elif ((df['RFM_Score_sum'] >= 12) and (df['RFM_Score_sum'] < 14)):
        return 'Champions'
    elif ((df['RFM_Score_sum'] >= 10) and (df['RFM_Score_sum'] < 12)):
        return 'Loyal'
    elif ((df['RFM_Score_sum'] >= 8) and (df['RFM_Score_sum'] < 10)):
        return 'Potential'
    elif ((df['RFM_Score_sum'] >= 6) and (df['RFM_Score_sum'] < 8)):
        return 'Promising'
    elif ((df['RFM_Score_sum'] >= 4) and (df['RFM_Score_sum'] < 6)):
        return 'Needs Attention'
    else:
        return 'Require Activation'# Create a new variable RFM_Level

df_rfm_segmented['RFM_Level'] = df_rfm_segmented.apply(rfm_level, axis=1)# Print the header with top 5 rows to the console
df_rfm_segmented.head()

# df_rfm_segmented.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/rfm_segmented.xlsx')

In [ ]:
## Box plot for Recency distributions at segment levels  : 

 
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context("paper", rc={"font.size":15,"axes.titlesize":25,"axes.labelsize":35})  
sns.set(font_scale = 1.5)
sns.set_style('white')


sns.boxplot(y="recency", x="RFM_Level",data=df_rfm_segmented)
plt.title("Distribution of recency accross different segments of customers who bought more than once")
plt.xticks(rotation=60)

In [ ]:
##Finding top 10 customers of top two segments : 

df_rfm_filtd=df_rfm_segmented[df_rfm_segmented['RFM_Level'].isin(['Can\'t Loose Them','Champions'])]

df_rfm_top_users=df_rfm_filtd.groupby(['RFM_Level','username_proscsd']).agg({'RFM_Score_sum':'max'}).sort_values('RFM_Score_sum').reset_index()

df_top_10_users=df_rfm_top_users.sort_values('RFM_Score_sum',ascending = False).groupby('RFM_Level').head(10)

top_10_cust_list=df_top_10_users['username_proscsd'].tolist()

df_cantloose_champ= df_rfm_filtd[df_rfm_filtd['username_proscsd'].isin(top_10_cust_list)].sort_values(['RFM_Level','RFM_Score_sum'])


df_cantloose_champ.head()

# df_cantloose_champ.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/df_cantloose_champ.xlsx')

In [ ]:
## Segment wise analysis and creating customer profiles with value , volume , recency , frequency , price etc : 

df_segmented_summary = df_rfm_segmented.groupby('RFM_Level').agg({'Total':'sum','Lineitem quantity':'sum',\
                                                                  'recency':['mean','median'],'frequency':['mean','median'],\
                                                                 'Name':pd.Series.nunique,'Product_Category':pd.Series.mode,\
                                                                 'Lineitem price': ['mean','median'],\
                                                                 'Shipping Zip':pd.Series.mode,\
                                                                 'username_proscsd':pd.Series.nunique,'RFM_Score_sum': ['max','min']}).reset_index().\
rename(columns={'Total':'Sales Value','Lineitem quantity':'Sales Volume','Name':'#Transactions','username_proscsd':'#Unique Customers'})


df_segmented_summary

df_bubblechart=df_segmented_summary[['RFM_Level','Sales Value','Sales Volume','#Transactions','frequency','recency','#Unique Customers']]
 
df_bubblechart.columns = ['_'.join(col) for col in df_bubblechart.columns.values]


df_bubblechart=df_bubblechart.rename(columns={'RFM_Level_':'RFM Segment'})

df_bubblechart.head()

# df_segmented_summary.head()

# df_segmented_summary.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/rfm_segmented_summary.xlsx')

In [ ]:
##Visualizing the different segments in a bubble chart and further possibilities of clubbing few segments together : 

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context("paper", rc={"font.size":15,"axes.titlesize":25,"axes.labelsize":35})  
sns.set(font_scale = 1.5)
sns.set_style('white')


g=sns.scatterplot(data=df_bubblechart, x="Sales Value_sum", y="frequency_mean", size="#Transactions_nunique", hue="RFM Segment",alpha=0.5,legend='brief',sizes=(20, 2000))

h,l = g.get_legend_handles_labels()
plt.legend(h[0:7],l[0:7],bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel("Sales Value")
plt.ylabel("Mean Frequency")
plt.show(g)



In [ ]:
## RFM Segment wise unique customer counts : 

ax = sns.barplot(x="RFM Segment", y="#Unique Customers", data=df_bubblechart)
ax.set(ylabel="Count of customers")

plt.xticks(rotation=60)
plt.show()


In [ ]:
## analyiss and profiling for customers who bought only once : 

from datetime import date
today = datetime.now()

df_segment_1=df_seg2[df_seg2['#Transactions']==1]
df_segment_1['Date_copy']=df_segment_1['Date']

rfm_df_1= df_segment_1.groupby('username_proscsd').agg({'Date': lambda date: (today - date.max()).days,
                                        'Date_copy': lambda Date: Date.nunique(),
                                        'Total': lambda Total: Total.sum(),\
                                        'Lineitem quantity':lambda Total: Total.sum()}).reset_index()

rfm_df_1.columns=['username_proscsd','recency','frequency','monetary','Total Qty']
df_segment_1_merged=df_segment_1.merge(rfm_df_1,on='username_proscsd',how='left')
# rfm_df['monetary']=rfm_df['monetary']/rfm_df['frequency']

df_segment_1_merged.head()

df_segment_1_merged.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/df_segment_1_merged.xlsx')


In [ ]:
# Plot RFM distributions for sigle purchase customers : 
import matplotlib.pyplot as plt
import seaborn as sns
# plt.figure(figsize=(12,10))# Plot distribution of R
plt.subplot(3, 1, 1); sns.distplot(rfm_df_1['recency']).set_title('Recency')# Plot distribution of F
plt.subplot(3, 1, 2); sns.distplot(rfm_df_1 ['frequency']).set_title('Frequency')# Plot distribution of M
plt.subplot(3, 1, 3); sns.distplot(rfm_df_1 ['monetary'])# Show the plot
plt.show()

# sns.boxplot(y=["recency","frequency","monetary"], data=rfm_df_1)
sns.boxplot(x="variable", y="value", data=pd.melt(rfm_df_1))


# <font color='red'>C. Market Basket Analysis :  </font>

In [ ]:
##Basket Distribution :
## Customer wise finding no of qtys. :


df_filtd_colmns['Lineitem_name_processed']=df_filtd_colmns['Lineitem name'].str.lower().apply(lambda x: x.strip())

df_full= df_filtd_colmns[df_filtd_colmns['Financial Status']=='paid']


df_basket=df_full.groupby(['Name']).agg({'Lineitem_name_processed': pd.Series.nunique,'Lineitem quantity':'sum'}).reset_index().rename(columns={'Lineitem_name_processed': 'Basket_Size'})


df_basket_filtd= df_basket[df_basket['Basket_Size']>1]
print(len(df_basket_filtd))
df_basket_filtd.head()

Trans_list_grt1 = df_basket_filtd['Name'].tolist()
print(len(Trans_list_grt1))

df_basket_filtd_colmns= df_full[df_full.Name.isin(Trans_list_grt1)]
print(len(df_basket_filtd_colmns))



In [ ]:
df_grouped_trans=df_basket_filtd_colmns.groupby('Name')['Lineitem_name_processed'].apply(list).reset_index(name='Itemsets')
transactions=df_grouped_trans['Itemsets'].tolist()

In [ ]:
df_basket_total=df_basket_filtd_colmns[['Name','Lineitem_name_processed','Lineitem quantity']]
# df_item_sku_map=pd.DataFrame(df_full.groupby(['Lineitem name']).size().reset_index(name='Freq'))

In [ ]:
market_basket = df_basket_total.groupby(
                ['Name', 'Lineitem_name_processed'])['Lineitem quantity']
market_basket = market_basket.sum().unstack().reset_index().fillna(0).set_index('Name')

market_basket['Basket_Size']= market_basket.sum(axis = 1, skipna = True)
market_basket_filtd=market_basket[market_basket['Basket_Size']>1]
market_basket_filtd.head()

In [ ]:

market_basket_filtd=market_basket_filtd.drop('Basket_Size',axis=1)
print(len(market_basket))
print(len(market_basket_filtd))
market_basket_filtd.head()

In [ ]:
## One hot encoding of the different SKUs to understand which are bought tohgether. 

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = market_basket_filtd.applymap(encode_units)
# basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets.head()

In [ ]:
#Getting the list of transactions from the dataset
transactions = []
for i in range(0, len(set(Trans_list_grt1))):
    transactions.append([str(market_basket_filtd.values[i,j]) for j in range(0, 2954)])
    
df_check=df_basket_filtd_colmns.groupby('Name')['Lineitem_name_processed'].apply(list).reset_index(name='Itemsets')

In [ ]:
df_grouped_trans=df_basket_filtd_colmns.groupby('Name')['Lineitem_name_processed'].apply(list).reset_index(name='Itemsets')


transactions=df_grouped_trans['Itemsets'].tolist()

In [ ]:
# Training Apriori algorithm on the dataset
# https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python
from apyori import apriori
rule_list = apriori(transactions, min_support = 0.003, min_confidence = 0.3, min_lift = 3, min_length = 2)

In [ ]:
for item in rule_list:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

In [ ]:
#Now that the data is structured properly, we can generate frequent item sets that have a support of at least 7%

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = fpgrowth(basket_sets, min_support=0.007,use_colnames=True)

frequent_itemsets
# frequent_itemsets.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/frequent_itemsets.xlsx')

In [ ]:
items = df_basket_filtd_colmns.groupby(['Name'])\
.agg({'Lineitem_name_processed': lambda x: x.ravel().tolist()}).reset_index().rename(columns={'Lineitem_name_processed':'itemsets'})

items.head()


In [ ]:
import itertools
import pandas as pd

combinations_list = []
for row in items.itemsets:
    combinations = list(itertools.combinations(row, 2))
    combinations_list.append(combinations)

combinations_list[:2]

combination_counts = pd.Series(combinations_list).explode().reset_index(drop=True)

combination_counts[:5]
combination_counts.value_counts()[:10]

In [ ]:
df_combination= pd.DataFrame(combination_counts.value_counts())


# df_combination.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/mktbskt_comb.xlsx')

In [ ]:
# Training Apriori algorithm on the dataset
rule_list = apriori(transactions, min_support = 0.003, min_confidence = 0.3, min_lift = 3, min_length = 2)

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.0000001)
rules

In [ ]:
df_full= df_filtd_colmns[df_filtd_colmns['Financial Status']=='paid']
df_full['username_proscsd']=df_full['username'].str.lower()

# df_check=df_full.groupby(['username_proscsd','Name'])['Date'].first()
# df_check.head()

df_full_sorted=df_full.sort_values(['username_proscsd', 'Name','Date'], ascending=[True, True,True])


df_full_sorted.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/df_full_sorted.xlsx')
df_full_sorted.head()

In [ ]:
df_trans=df_full_sorted.groupby(['username_proscsd','Name','Date']).agg(Total_qty=('Lineitem quantity','sum'),\
                                                                         Item_List=('Lineitem_name_processed', \
                                                                         lambda x: x.ravel().tolist()),\
                                                                        Unq_qty=('Lineitem_name_processed',lambda x: x.nunique())).reset_index()

df_trans_sorted=df_trans.sort_values(['username_proscsd', 'Name','Date'], ascending=[True, True,True])

df_trans_sorted['#Purchase_sequence']=df_trans_sorted.groupby(['username_proscsd']).cumcount()+1
df_trans_sorted.head()
# df_trans_sorted.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/df_trans_sorted.xlsx')

In [ ]:
df_trans_1_item=df_trans_sorted[df_trans_sorted['#Purchase_sequence']==1]['Item_List'].tolist()
df_trans_1_item_list=[str(item[0]) for item in df_trans_1_item]
df_trans_1_item_df=pd.DataFrame(pd.Series(df_trans_1_item_list).value_counts()).reset_index()
df_trans_1_item_df.columns=['SKU Name','Count']
df_trans_1_item_df['Percentage Occurance'] = round(100*df_trans_1_item_df['Count']/df_trans_1_item_df['Count'].sum(),2)
df_trans_1_item_df['#Sequence']=1


df_trans_2_item=df_trans_sorted[df_trans_sorted['#Purchase_sequence']==2]['Item_List'].tolist()
df_trans_2_item_list=[str(item[0]) for item in df_trans_2_item]
df_trans_2_item_df=pd.DataFrame(pd.Series(df_trans_2_item_list).value_counts()).reset_index()
df_trans_2_item_df.columns=['SKU Name','Count']
df_trans_2_item_df['Percentage Occurance'] = round(100*df_trans_2_item_df['Count']/df_trans_2_item_df['Count'].sum(),2)
df_trans_2_item_df['#Sequence']=2

df_trans_3_item=df_trans_sorted[df_trans_sorted['#Purchase_sequence']==3]['Item_List'].tolist()
df_trans_3_item_list=[str(item[0]) for item in df_trans_3_item]
df_trans_3_item_df=pd.DataFrame(pd.Series(df_trans_3_item_list).value_counts()).reset_index()
df_trans_3_item_df.columns=['SKU Name','Count']
df_trans_3_item_df['Percentage Occurance'] = round(100*df_trans_3_item_df['Count']/df_trans_3_item_df['Count'].sum(),2)
df_trans_3_item_df['#Sequence']=3


df_trans_4_item=df_trans_sorted[df_trans_sorted['#Purchase_sequence']==4]['Item_List'].tolist()
df_trans_4_item_list=[str(item[0]) for item in df_trans_4_item]
df_trans_4_item_df=pd.DataFrame(pd.Series(df_trans_4_item_list).value_counts()).reset_index()
df_trans_4_item_df.columns=['SKU Name','Count']
df_trans_4_item_df['Percentage Occurance'] = round(100*df_trans_4_item_df['Count']/df_trans_4_item_df['Count'].sum(),2)
df_trans_4_item_df['#Sequence']=4


df_trans_final_top10= pd.concat([df_trans_1_item_df[:10],df_trans_2_item_df[:10],df_trans_3_item_df[:10],df_trans_4_item_df[:10]],axis=0).reset_index(drop=True)

df_trans_final_top10.head(2)

df_trans1_wide=df_trans_1_item_df.loc[:10,['SKU Name','Percentage Occurance','#Sequence']]
df_trans2_wide=df_trans_2_item_df.loc[:10,['SKU Name','Percentage Occurance','#Sequence']]
df_trans3_wide=df_trans_3_item_df.loc[:10,['SKU Name','Percentage Occurance','#Sequence']]
df_trans4_wide=df_trans_4_item_df.loc[:10,['SKU Name','Percentage Occurance','#Sequence']]


df_trans_final_wide= pd.concat([df_trans1_wide,df_trans2_wide,df_trans3_wide,df_trans4_wide],axis=1)
df_trans_final_wide.head()


# df_trans_final_wide.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/df_trans_final_wide.xlsx')

In [ ]:
##Top 10 selling SKUs of each year : 
df_full= df_filtd_colmns[df_filtd_colmns['Financial Status']=='paid']
# df_full2= df_full[df_full['year']!='2016'].copy()
df_top_sku=df_full.groupby(['year','Lineitem_name_processed']).agg({'Total':'sum'}).reset_index().\
                                                                sort_values(['year','Total'], ascending=[False,False])

# df_top_sku['Percentage Sales'] =df_top_sku.groupby('year').agg({lambda x:x round(100*df_top_sku['Total']/df_top_sku['Total'].sum(),2)})                                   

f = lambda x: 100 * x / float(x.sum())
df_top_sku['perc_sales'] = (df_top_sku.groupby('year')['Total']
                                        .transform(f))

df_top_sku.head()

# df_full.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/processed_sales_data_fnl.xlsx')

In [ ]:
df_full['year_numeric']=df_full['year'].astype(int)


df_stacked_top_10=pd.DataFrame()

f = lambda x:  x / float(x.sum())
df_top_sku['perc_sales'] = (df_top_sku.groupby('year')['Total']
                                        .transform(f))


for year in (2017,2018,2019,2020):
    
    df=pd.DataFrame()
    df_top_sku=pd.DataFrame()
    df_top_10_sku=pd.DataFrame()
    
    df=df_full[df_full['year_numeric']==year]
    
    df_top_sku=df.groupby(['year','Lineitem_name_processed']).agg({'Total':'sum'}).reset_index().\
                                                                sort_values('Total', ascending=False)
    df_top_sku['perc_sales'] = (df_top_sku.groupby('year')['Total']
                                        .transform(f))

    
    
    df_top_10_sku=df_top_sku[:10]
    

    
    df_stacked_top_10=pd.concat([df_stacked_top_10,df_top_10_sku],axis=0)
    

df_stacked_top_10

df_stacked_top_10.to_excel('C:/Users/KINSUK/Desktop/DS_Freelancing/RTB_Analyttica/Acai_Sales_Analysis_May2021/df_stacked_top_10.xlsx')